In [1]:
from pipeline02 import *

In [2]:
wandb.login()

wandb: Currently logged in as: teamtom (use `wandb login --relogin` to force relogin)


True

In [6]:
sweep_config = {
    'method': 'grid',
    'metric': {
        'name': 'f1',
        'goal': 'maximize'   
        }
    }

parameters_dict = {
    'epochs': {
        'value' : 100
        },
    'learning_rate': {
        'values': [0.01, .005, 0.001, .005, 0.0001]
        },
    'batch_size': {
        'value': 32
        },
    'threshold': {
        'values': [0.2, 0.21578947368421053, 0.23157894736842105, 0.2473684210526316, 0.26315789473684215, 0.2789473684210526, 0.2947368421052632, 
                 0.3105263157894737, 0.3263157894736842, 0.34210526315789475, 0.35789473684210527, 0.37368421052631584, 0.3894736842105263, 
                 0.4052631578947369, 0.4210526315789474, 0.4368421052631579, 0.45263157894736844, 0.46842105263157896, 0.4842105263157895, 0.5]
        }
    }

sweep_config['parameters'] = parameters_dict

sweep_id = wandb.sweep(sweep_config, project="churn-class-imbalance-with-treshold01")

Create sweep with ID: d22sfkpk
Sweep URL: https://wandb.ai/teamtom/churn-class-imbalance-with-treshold01/sweeps/d22sfkpk


In [7]:
df = pd.read_csv('..\Churn_Modelling.csv')

df.Gender.replace({'Female':0, 'Male':1}, inplace=True)

# class imbalance
class0_num = df[df.Exited==0]['Exited'].count()
class1_num = df[df.Exited==1]['Exited'].count()
print(class0_num, class1_num)
print(class0_num / class1_num)

df_inputs = df[['CreditScore', 'Gender', 'Age','Tenure','Balance','NumOfProducts','HasCrCard', 'IsActiveMember','EstimatedSalary']]
df_targets = df[['Exited']]

7963 2037
3.9091801669121256


In [8]:
pl = Pipeline(task='classification', optimizer=torch.optim.Adam, loss_fn=F.binary_cross_entropy, wandb_log=True )

In [9]:
pl.device

device(type='cpu')

In [10]:
def sweep_me(config=None):
    
    with wandb.init(config=config):
        config = wandb.config
        
        pl.threshold = config.threshold
        pl.epochs = config.epochs
        pl.learning_rate = config.learning_rate
        pl.batch_size = config.batch_size

        pl.load_tabular_data(df_inputs.values, df_targets.values, split=[0.9, 0.9], stratify=True)
        
        model = nn.Sequential(
            nn.Linear(9,1),
            # nn.Linear(14,32),
            # nn.ReLU(),
            # nn.Linear(32,16),
            # nn.ReLU(),
            # nn.Linear(16,1),
            nn.Sigmoid()
        ).to(pl.device)
        
        history = pl.fit(model, show_nth_epoch=10)
        
        acc, precision, recall, f1 = pl.show_metrics(model, pl.test_loader, supress_print=True)
        
        wandb.run.summary["accuracy"] = acc
        wandb.run.summary["prec"] = precision
        wandb.run.summary["recall"] = recall
        wandb.run.summary["f1"] = f1

In [11]:
wandb.agent(sweep_id, sweep_me)

wandb: Agent Starting Run: 6o4tq9js with config:
wandb: 	batch_size: 32
wandb: 	epochs: 100
wandb: 	learning_rate: 0.01
wandb: 	threshold: 0.2
wandb: wandb version 0.12.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


dataset size: 10000
dataset samples shape: torch.Size([10000, 9])
dataset classes shape: torch.Size([10000, 1])
| Epoch: 0010 | Train Loss: 0.440 | Train Acc: 66.87% | Val. Loss: 0.454 | Val. Acc: 65.52% |
| Epoch: 0020 | Train Loss: 0.441 | Train Acc: 67.78% | Val. Loss: 0.427 | Val. Acc: 65.09% |
| Epoch: 0030 | Train Loss: 0.438 | Train Acc: 67.89% | Val. Loss: 0.416 | Val. Acc: 72.63% |
| Epoch: 0040 | Train Loss: 0.440 | Train Acc: 68.18% | Val. Loss: 0.420 | Val. Acc: 66.27% |
| Epoch: 0050 | Train Loss: 0.440 | Train Acc: 68.47% | Val. Loss: 0.446 | Val. Acc: 70.80% |
| Epoch: 0060 | Train Loss: 0.438 | Train Acc: 67.80% | Val. Loss: 0.424 | Val. Acc: 64.55% |
| Epoch: 0070 | Train Loss: 0.439 | Train Acc: 67.94% | Val. Loss: 0.422 | Val. Acc: 66.59% |
| Epoch: 0080 | Train Loss: 0.439 | Train Acc: 68.33% | Val. Loss: 0.430 | Val. Acc: 68.32% |
| Epoch: 0090 | Train Loss: 0.439 | Train Acc: 68.59% | Val. Loss: 0.427 | Val. Acc: 70.04% |
| Epoch: 0100 | Train Loss: 0.438 | Train 

train_loss,0.43812
train_acc,0.67913
valid_loss,0.43221
valid_acc,0.67349
lr,0.01
epoch_time,0.20322
_runtime,31
_timestamp,1633290589
_step,99
accuracy,0.671
prec,0.3494


train_loss,█▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▆▇▇▇███████████████████████████████████
valid_loss,█▃▃▂▂▁▂▃▂▂▃▃▁▂▃▁▁▁▃▁▂▄▁▄▁▂▃▁▄▂▃▁▁▄▃▂▁▂▃▃
valid_acc,▁▃▆▆▆▇▇▇▆▆▆▇▆▆▇▇▇▇▆▇█▆▇▅▇▇▇▇▇▇▇▇▇▇█▇▇▆▇▆
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch_time,▅▇▆▆▇▇▆▆▆▅▄▆▆▆▇▄▆▆▆▇▆▆▆▇▄▇▄▆█▇▄▆▇▃▁▂▄▂▂▃
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇██
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇██
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


wandb: Agent Starting Run: q3p35031 with config:
wandb: 	batch_size: 32
wandb: 	epochs: 100
wandb: 	learning_rate: 0.01
wandb: 	threshold: 0.21578947368421053
wandb: wandb version 0.12.3 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


...

dataset size: 10000
dataset samples shape: torch.Size([10000, 9])
dataset classes shape: torch.Size([10000, 1])
| Epoch: 0010 | Train Loss: 0.529 | Train Acc: 79.40% | Val. Loss: 0.517 | Val. Acc: 79.85% |
| Epoch: 0020 | Train Loss: 0.498 | Train Acc: 79.70% | Val. Loss: 0.484 | Val. Acc: 80.28% |
| Epoch: 0030 | Train Loss: 0.494 | Train Acc: 79.70% | Val. Loss: 0.489 | Val. Acc: 79.53% |
| Epoch: 0040 | Train Loss: 0.492 | Train Acc: 79.61% | Val. Loss: 0.483 | Val. Acc: 79.53% |
| Epoch: 0050 | Train Loss: 0.488 | Train Acc: 79.61% | Val. Loss: 0.471 | Val. Acc: 80.28% |
| Epoch: 0060 | Train Loss: 0.485 | Train Acc: 79.70% | Val. Loss: 0.481 | Val. Acc: 78.77% |
| Epoch: 0070 | Train Loss: 0.482 | Train Acc: 79.70% | Val. Loss: 0.467 | Val. Acc: 80.28% |
| Epoch: 0080 | Train Loss: 0.481 | Train Acc: 79.61% | Val. Loss: 0.472 | Val. Acc: 79.53% |
| Epoch: 0090 | Train Loss: 0.479 | Train Acc: 79.64% | Val. Loss: 0.461 | Val. Acc: 80.28% |
| Epoch: 0100 | Train Loss: 0.476 | Train 

C:\Users\teamt\AppData\Roaming\Python\Python38\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


train_loss,0.4758
train_acc,0.79737
valid_loss,0.47137
valid_acc,0.78772
lr,0.0001
epoch_time,0.21328
_runtime,30
_timestamp,1633293379
_step,99
accuracy,0.796
prec,0.0


train_loss,█▆▄▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▅▇█████████████████████████████████████
valid_loss,█▆▄▃▃▂▂▂▂▂▂▂▂▁▂▁▂▂▁▂▁▂▂▁▂▁▁▂▂▁▁▁▁▁▁▁▁▂▁▁
valid_acc,▁▅▇██████▇██████████████████████████████
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch_time,▁▄▂▃▃▃▃▃▃▃▃▃▃▃▄▃▃▃▃▄▃▃▄▄▃▃█▃▄▄▂▁▂▂▂▁▂▂▂▂
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇▇▇██
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇▇▇██
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
